In [1]:
import arcpy.management
from arcgis.gis import GIS
from arcgis.raster import ImageryLayer
from arcgis.raster.functions import apply


import requests
import zipfile
import os
import tempfile
import datetime
import time
import arcpy
import hashlib
import arcpy


C:\Users\rami8629\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# Functions
<font color="red">**CAUTION**: Modify the functions below only if you understand the consequences.</font>

In [2]:
def download_and_extract(year):
    # Download the zipped file
    url = f"https://sentinel.esa.int/documents/d/sentinel/sentinel-2a-acquisition-plans-{year}"
    response = requests.get(url)

    # Save the zipped file to a temporary directory
    temp_dir = tempfile.mkdtemp()
    zip_path = os.path.join(temp_dir, "sentinel.zip")
    with open(zip_path, "wb") as f:
        f.write(response.content)

    # Extract the zipped file
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(temp_dir)

    return temp_dir
def calculate_size(cell_size, max_pixels):
    # Calculate the size in square meters
    size_sqm = (cell_size * max_pixels)**2

    # Convert the size to square kilometers
    size_sqkm = size_sqm / 1e6

    return size_sqkm
def create_tessellation(aoi, cell_size, max_pixels):

    # Create the tessellation
    tessellation = arcpy.GenerateTessellation_management(
        Output_Feature_Class=r"memory/tessellation",
        Extent=aoi,
        Shape_Type="SQUARE",
        Size =  f"{calculate_size(cell_size, max_pixels)} SquareKilometers"

    )

    # Initialize the list of extents
    extents = []

    # Assume tessellation is the output from the create_tessellation function
    with arcpy.da.SearchCursor(tessellation, ["SHAPE@"]) as cursor:
        for row in cursor:
            feature = row[0]
            extent = feature.extent

            # Add the extent to the list
            extents.append(f'{extent.XMin},{extent.YMin},{extent.XMax},{extent.YMax}')

    return tessellation


def parse_kml_files(directory, tessellation, start_date, end_date):
    # Initialize the list of acquisition dates
    acq_dates_extent = []
    kmlgdbs = []
    # Loop through the files in the directory
    for filename in os.listdir(directory):
        if filename.endswith(".kml"):
            # Extract the acquisition dates from the KML file name
            acq_start, acq_end = filename.split("_")[5:7]
            acq_start = datetime.datetime.strptime(acq_start, "%Y%m%dT%H%M%S")
            acq_end = datetime.datetime.strptime(acq_end.split(".")[0], "%Y%m%dT%H%M%S")
            # Check if the acquisition dates fall within the defined start and end dates
            if start_date <= acq_start <= end_date or start_date <= acq_end <= end_date:
                # Convert the KML to a Layer
                arcpy.env.workspace = directory

                arcpy.conversion.KMLToLayer(os.path.join(directory, filename), directory)

                # Loop through all the file geodatabases in the workspace
                gdb = [gdb for gdb in arcpy.ListWorkspaces('*', 'FileGDB') if gdb not in kmlgdbs][0]
                kmlgdbs.append(gdb)

                arcpy.env.workspace = gdb

                # Find the polygon feature class in the file geodatabase
                
                datasets = arcpy.ListDatasets(feature_type='feature')
                datasets = [''] + datasets if datasets is not None else []

                for ds in datasets:
                    for fc in arcpy.ListFeatureClasses(feature_dataset=ds):
                        if arcpy.Describe(fc).shapeType == "Polygon":
                            # Perform a spatial join with the input AOI
                            arcpy.analysis.SpatialJoin(
                                target_features=tessellation,
                                join_features=f"{ds}\\{fc}",
                                out_feature_class="join",
                                join_operation="JOIN_ONE_TO_MANY",
                                join_type="KEEP_ALL",
                                field_mapping=None,
                                match_option="INTERSECT",
                                search_radius=None,
                                distance_field_name="",
                                match_fields=None
                            )
                            # Open a search cursor on the in_memory output
                            with arcpy.da.SearchCursor("join", ["BeginTime", "EndTime", "SHAPE@"], "Join_Count = 1") as cursor:
                                for row in cursor:
                                    extent_start_date = row[0].split("T")[0]
                                    start_of_date_unix = int(time.mktime(datetime.datetime.strptime(extent_start_date, '%Y-%m-%d').timetuple())) * 1000
                                    extent_end_date = row[1].split("T")[0]
                                    end_of_date_unix = int(((time.mktime(datetime.datetime.strptime(extent_end_date, '%Y-%m-%d').timetuple())) * 1000) + (43200000*1.5))
                                    # Append the start date of the filtered features to the acq_dates list
                                    acq_dates_extent.append([f"{start_of_date_unix}, {end_of_date_unix}", f"{row[2].extent.XMin},{row[2].extent.YMin},{row[2].extent.XMax},{row[2].extent.YMax}"])
                                del cursor, row
                        arcpy.management.Delete("join")
                            
                arcpy.env.workspace = directory

    return acq_dates_extent

def download_image(username, password, acq_date, max_pixels, extent, output_dir):
    gis = GIS("https://www.arcgis.com", username, password)
    sentinel_item=gis.content.search('255af1ceee844d6da8ef8440c8f90d00', 'Imagery Layer', outside_org=True)[0]
    s2_layer = sentinel_item.layers[0]

    sentinel_AllBands = apply(s2_layer, 'None')

    # Convert the acquisition dates from Unix timestamps to the ddMMYYYY format
    acq_start, acq_end = acq_date.split(", ")
    acq_start = datetime.datetime.fromtimestamp(int(acq_start) / 1000).strftime("%d%m%Y")
    acq_end = datetime.datetime.fromtimestamp(int(acq_end) / 1000).strftime("%d%m%Y")

    # Generate a unique hash for the extent
    extent_hash = hashlib.md5(str(extent).encode()).hexdigest()
    
    out_file = f"Sentinel2_{acq_start}_{acq_end}_{extent_hash}.tif"

    if out_file in os.listdir(output_dir):
        return
    

    # Define the output filename
    out_file = f"Sentinel2_{acq_start}_{acq_end}_{extent_hash}.tif"
    print(out_file)
    # Export the image
    sentinel_AllBands.export_image(bbox=extent, 
                        save_folder=output_dir, 
                        save_file=out_file, 
                        image_sr=3857, 
                        bbox_sr=3857, 
                        size=[max_pixels, max_pixels],
                        time=acq_date,
                        f="image", 
                        export_format="tiff")
def process_raster_data(output_dir):
    with arcpy.EnvManager(outputCoordinateSystem='PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]'):
        outgdb = os.path.join(output_dir, "sentinel2.gdb")
        arcpy.management.CreateFileGDB(output_dir, "sentinel2.gdb")

        arcpy.management.CreateMosaicDataset(
            in_workspace=outgdb,
            in_mosaicdataset_name="sentinel2",
            coordinate_system='PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]',
            num_bands=None,
            pixel_type="",
            product_definition="",
            product_band_definitions=None
        )
        arcpy.management.AddRastersToMosaicDataset(
            in_mosaic_dataset=os.path.join(outgdb, "sentinel2"),
            raster_type="Raster Dataset",
            input_path=f"{output_dir}",
            update_cellsize_ranges="UPDATE_CELL_SIZES",
            update_boundary="UPDATE_BOUNDARY",
            update_overviews="UPDATE_OVERVIEWS",
            maximum_pyramid_levels=None,
            maximum_cell_size=0,
            minimum_dimension=1500,
            spatial_reference=None,
            filter="",
            sub_folder="SUBFOLDERS",
            duplicate_items_action="ALLOW_DUPLICATES",
            build_pyramids="BUILD_PYRAMIDS",
            calculate_statistics="CALCULATE_STATISTICS",
            build_thumbnails="BUILD_THUMBNAILS",
            operation_description="",
            force_spatial_reference="NO_FORCE_SPATIAL_REFERENCE",
            estimate_statistics="ESTIMATE_STATISTICS",
            aux_inputs=None,
            enable_pixel_cache="NO_PIXEL_CACHE",
        )
        
    arcpy.management.MakeRasterLayer(os.path.join(outgdb, "sentinel2"), "sentinel2")

    # Define the feature class
    fc = "sentinel2"

    # Add the "AquisitionDate" field
    arcpy.AddField_management(fc, "AquisitionDate", "DATE", field_alias="Aquisition Date")

    # Define the update cursor
    cursor = arcpy.da.UpdateCursor(fc, ["Name", "AquisitionDate"])
    for row in cursor:
        if "Sen" in row[0]:
            # Extract the date string from the field value
            date_str = row[0].split("_")[2]

            # Convert the date string to a date object
            date_obj = datetime.datetime.strptime(date_str, "%d%m%Y").date()

            # Update the "AquisitionDate" field
            row[1] = date_obj

            # Update the row
            cursor.updateRow(row)
    del cursor, row
    return os.path.join(outgdb, "sentinel2")


# Variables

In [3]:

# Define Area of Interest (AOI) and output directory
# Change the aoi to the path of the feature class or shapefile that represents the area of interest
aoi = r"E:\OneDrive - Esri\Demos & Blogs\ArcGIS Resources\GeoAi & Deep Learning\Demos\Biomass_Cantabria\Biomass_Cantabria.gdb\Cantabria_Boundaries_WGS1984_Pro"
# Change the output_dir to the directory where you want to save the downloaded images and the processed raster data
output_dir = r"E:\OneDrive - Esri\Demos & Blogs\ArcGIS Resources\GeoAi & Deep Learning\Demos\Biomass_Cantabria\Sentinel2_15Bands"
arcpy.env.overwriteOutput = True
#Input AGOL credentials
username = input("Enter your ArcGIS Online username: ")
password = input("Enter your ArcGIS Online password: ")

# Define the start and end dates
start_date = datetime.datetime(2023, 5, 1)
end_date = datetime.datetime(2023, 6, 30)

# Define the maximum number of pixels
pixel_size = 256

# Download and create mosaic

**DISCLAIMER**: Running the following cell may take <font color="green">**minutes to hours to sometimes days**</font> depending on the size of your <font color="green">**Area of Interest (AOI)**</font> and the range of your <font color="green">**start and end dates**</font>. Please be patient and allow the process to complete.

In [4]:

years = list(range(start_date.year, end_date.year+1))

for year in years:
    temp_dir = download_and_extract(year)

tessellation = create_tessellation(aoi, 10, 256)

acq_dates_extent = parse_kml_files(temp_dir, tessellation, start_date, end_date)

# Convert inner lists to tuples and remove duplicates
unique_acq_dates_extent = list(set(tuple(x) for x in acq_dates_extent))

# Convert inner tuples back to lists
unique_acq_dates_extent = [list(x) for x in unique_acq_dates_extent]

# Calculate the total number of images to be downloaded
total_images = len(unique_acq_dates_extent)
print(f"Total images to be downloaded: {total_images}")


# Loop through the list of acquisition dates and extents
image_count = 0
for acq_date_extent in unique_acq_dates_extent:
    download_image(username, password, acq_date_extent[0], 256, acq_date_extent[1], output_dir)
    image_count += 1
    print(f"Downloaded {image_count} of {total_images} images. {total_images - image_count} images left.")

# Process the raster data
out_mosaic = process_raster_data(output_dir)

print(f"Processed raster data saved to {out_mosaic}")


Total images to be downloaded: 39150
Sentinel2_07062023_07062023_890bf9c31e8f6d6e3ac39281f3812313.tif
Downloaded 1 of 39150 images. 39149 images left.
Sentinel2_10072023_10072023_b62e678f6b99a814d7afb30453000285.tif
Downloaded 2 of 39150 images. 39148 images left.
Sentinel2_21042023_21042023_7c5ca6507d85576b20f5acf8a46e7641.tif
Downloaded 3 of 39150 images. 39147 images left.
Sentinel2_31052023_31052023_263f104f08d47a06098979cd4d48ecdc.tif
Downloaded 4 of 39150 images. 39146 images left.
Sentinel2_20062023_20062023_2414d308be892d1e5f03411c045cf883.tif
Downloaded 5 of 39150 images. 39145 images left.
Sentinel2_31052023_31052023_dfa508c31fcacd3adf07364711ebce6f.tif
Downloaded 6 of 39150 images. 39144 images left.
Sentinel2_11052023_11052023_d38517b4a2c5045704f236dab843c809.tif
Downloaded 7 of 39150 images. 39143 images left.
Sentinel2_21052023_21052023_e1c93af8cbe8ce438500ca4092b28591.tif
Downloaded 8 of 39150 images. 39142 images left.
Sentinel2_11052023_11052023_d7e44557b9de8c9197ed231

ConnectionError: A connection error has occurred: HTTPSConnectionPool(host='esriaiddev.maps.arcgis.com', port=443): Max retries exceeded with url: /sharing/rest/search (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001CB5BBA09A0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))